In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deepface

In [ ]:

#replace file to mute warning
from deepface import DeepFace
import gdown
import os
import shutil
url = "https://drive.google.com/u/0/uc?id=1aS5SJCK9MLMyYPXmhAkwtn3cJhQGCbGj&export=download"
gdown.download(url, "/content/sample_data/DeepFace.py")
os.remove("/usr/local/lib/python3.7/dist-packages/deepface/DeepFace.py")
shutil.move("/content/sample_data/DeepFace.py","/usr/local/lib/python3.7/dist-packages/deepface/")


In [ ]:
from deepface import DeepFace
import gdown
import os
import shutil

In [ ]:
#!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_train_unmasked_folder.zip -d /content/sample_data
!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_train_masked.zip -d /content/sample_data

In [ ]:
from os import listdir
from os.path import join, isfile

In [ ]:
def alignfolder(unmasked_path,masked_path):
  files = listdir(unmasked_path)

  count = 0;
  for file in files:
    if not os.path.isfile(masked_path+"/"+file.split("_")[0]+".jpg"):
      print(file," not exist")
      os.remove(unmasked_path+"/"+file)
      count+=1

  print('total deleted :',count)    


alignfolder('/content/sample_data/AAFD_train_masked','/content/sample_data/AAFD_train_masked_folder')
#alignfolder('/content/sample_data/asia_celebA_test_unmasked','/content/sample_data/asia_celebA_test_masked')

test_unmasked = os.listdir('/content/sample_data/AAFD_train_masked')
test_masked = os.listdir('/content/sample_data/AAFD_train_unmasked_folder')
#train_unmasked = os.listdir('/content/sample_data/asia_celebA_train_unmasked')
#train_masked = os.listdir('/content/sample_data/asia_celebA_train_masked')



print("length test_unmasked :",len(test_unmasked))
print("length test_masked :",len(test_masked))
#print("length train_unmasked :",len(train_unmasked))
#print("length train_masked :",len(train_masked))

In [ ]:
#shutil.rmtree("/content/sample_data/result_face")
!mkdir /content/sample_data/result_face

In [ ]:
from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

EXPERIMENT_DATA_ARGS = {
    "unmasked_masked": {
        "model_path": "pretrained_models/unmasked_masked.pt",
        "image_path": "notebooks/images/input_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS["unmasked_masked"]
img_transforms = EXPERIMENT_ARGS['transform']
resize_dims = (256, 256)

In [ ]:
def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance  

In [ ]:

def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance 
N95 = 0
surgical = 0
cloth = 0
KN95 = 0

from torchvision.utils import save_image
from PIL import *
resize_dims = (256, 256)
masked_dir = "/content/sample_data/AAFD_train_masked/"
db_path = "/content/sample_data/AAFD_train_unmasked_folder"
count = 0
total_accuracy = 0
for masked_image_dir in os.listdir(masked_dir):
  if not masked_image_dir.endswith(".ipynb_checkpoints"):
    count += 1
    result_dir = masked_dir + masked_image_dir
    with torch.no_grad():
      tic = time.time()
      result_dir = masked_dir + masked_image_dir
      toc = time.time()
      df = DeepFace.find(img_path=result_dir,db_path=db_path,enforce_detection=False)
      accuracy = 1 - calc_distance(masked_image_dir.split(".")[0].split("_")[0],df)
      total_accuracy += accuracy
      if(accuracy == 0):
        mask_type = masked_image_dir.split("_")[1]
        if mask_type == "N95.jpg":
          N95 +=1
        elif mask_type == "KN95.jpg":
          KN95 +=1
        elif mask_type == "surgical.jpg":
          surgical +=1
        elif mask_type == "cloth.jpg":
          cloth +=1      

      print("masked_image :",masked_image_dir,"recognize_image:",df['identity'][0].split("/")[4],"Accuracy: ",accuracy)
      print("N95 :{} KN95 :{}  surgical :{} cloth:{} over image :{}".format(N95,KN95,surgical,cloth,count))

print("Accuracy of Model over {} images is : {}".format(count,total_accuracy/count))

In [ ]:
#ground_truth ffhq 0.6971544336001714